In [2]:
from typing import *
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Image, display, clear_output
import numpy as np
import seaborn as sns
import pandas as pd
sns.set_style("whitegrid")
import requests

import math
import torch
from torch import nn, Tensor
from torch.nn.functional import softplus
from torch.distributions import Distribution


from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from functools import reduce
from torch.distributions.bernoulli import Bernoulli

import gzip
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

## Helper functions

In [3]:
def load_data_chunk(filename, chunk_size=1000):
    """ Load a chunk of data from a gzipped TSV file. """
    return pd.read_csv(filename, sep='\t', compression='gzip', chunksize=chunk_size)

def separate_ids_and_data(data):
    ids = data.iloc[:, 0]
    data = data.iloc[:, 1:]
    return ids, data

class ReparameterizedDiagonalGaussian(Distribution):
    """
    A distribution `N(y | mu, sigma I)` compatible with the reparameterization trick given `epsilon ~ N(0, 1)`.
    """
    def __init__(self, mu: Tensor, log_sigma:Tensor):
        assert mu.shape == log_sigma.shape, f"Tensors `mu` : {mu.shape} and ` log_sigma` : {log_sigma.shape} must be of the same shape"
        self.mu = mu
        self.sigma = log_sigma.exp()

    def sample_epsilon(self) -> Tensor:
        """`\eps ~ N(0, I)`"""
        return torch.empty_like(self.mu).normal_()

    def sample(self) -> Tensor:
        """sample `z ~ N(z | mu, sigma)` (without gradients)"""
        with torch.no_grad():
            return self.rsample()

    def rsample(self) -> Tensor:
        """sample `z ~ N(z | mu, sigma)` (with the reparameterization trick) """
        # z = mu + sigma * epsilon
        return self.mu + self.sigma * self.sample_epsilon()

    def log_prob(self, z: Tensor) -> Tensor:
        """return the log probability: log `p(z)`"""
        # Log probability for Gaussian distribution
        # log p(z) = -1/2 * [log(2*pi) + 2*log(sigma) + (z - mu)^2/sigma^2]
        return -0.5 * (torch.log(2 * torch.tensor(math.pi)) + 2 * torch.log(self.sigma) +
                       torch.pow(z - self.mu, 2) / torch.pow(self.sigma, 2))
    
    def count_csv_rows(filename):
        # If the file is gzip-compressed, decompress it first
        if filename.endswith('.gz'):
            with gzip.open(filename, 'rt', newline='') as csvfile:
                row_count = sum(1 for row in csvfile)
        else:
            # Specify the correct encoding (e.g., 'utf-8', 'latin-1', etc.)
            encoding = 'utf-8'  # Change to the appropriate encoding if needed
            with open(filename, 'r', newline='', encoding=encoding) as csvfile:
                row_count = sum(1 for row in csvfile)
        return row_count

## Load Data

In [4]:
# Define the file paths
archs4_path = "/dtu-compute/datasets/iso_02456/archs4_gene_expression_norm_transposed.tsv.gz"
gtex_gene_path = "/dtu-compute/datasets/iso_02456/gtex_gene_expression_norm_transposed.tsv.gz"
gtex_isoform_path = "/dtu-compute/datasets/iso_02456/gtex_isoform_expression_norm_transposed.tsv.gz"
gtex_anno_path = "/dtu-compute/datasets/iso_02456/gtex_gene_isoform_annoation.tsv.gz"
gtex_tissue_path = "/dtu-compute/datasets/iso_02456/gtex_annot.tsv.gz"

num_genes = 18965
num_isoforms = 156958

num_genes, num_isoforms

(18965, 156958)

In [5]:
# count_csv_rows(gtex_gene_path)
print("gtex_gene_path num rows: 17357")
percentage_calc = 17357 * 0.0005
percentage_calc

gtex_gene_path num rows: 17357


8.6785

Using percentage 0.001 should load 17 samples, and 0.01 170. We use 0.005 for 86 samples

### Data Loader

- Label Encoding: This converts each unique label into a unique integer. If you have a large number of classes, be aware of memory usage.
- Inverse Transformation: If you need to get back the original labels from the encoded ones, you can use self.label_encoder.inverse_transform().
- Data Types: Labels are converted to torch.long since they are now integers, which is a common practice for categorical labels in classification tasks.

In [6]:
class GeneExpressionDataset(Dataset):
    """ # Old dataLoader with no TestSet
    def __init__(self, filepath, percentage=0.1):
        self.data = self.load_data_percentage(filepath, percentage)
        # Assuming the first column is the label
        self.labels = self.data.iloc[:, 0]
        self.genes = self.data.iloc[:, 1:]

        # Encode the labels
        self.label_encoder = LabelEncoder()
        self.encoded_labels = self.label_encoder.fit_transform(self.labels)
    """

    def __init__(self, data, labels):
        self.labels = labels
        self.genes = data
        self.label_encoder = LabelEncoder()
        self.encoded_labels = self.label_encoder.fit_transform(self.labels)

    def __len__(self):
        return len(self.genes)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        label = self.encoded_labels[idx]
        genes = self.genes.iloc[idx]
        return torch.tensor(genes.values, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

    def get_original_labels(self, encoded_labels):
        return self.label_encoder.inverse_transform(encoded_labels)

    @staticmethod
    def load_data_percentage(filepath, percentage=0.1):
        # Load a percentage of the data as shown previously
        cols = pd.read_csv(filepath, sep='\t', compression='gzip', nrows=0).columns
        n_total_rows = sum(1 for row in open(filepath, 'rb'))
        n_rows_to_load = int(n_total_rows * percentage)
        skip_rows = np.random.choice(np.arange(1, n_total_rows), size=n_total_rows - n_rows_to_load, replace=False)
        return pd.read_csv(filepath, sep='\t', compression='gzip', usecols=cols, skiprows=skip_rows)


In [7]:
load_percentage = 0.0005

# Load the entire dataset
full_data = GeneExpressionDataset.load_data_percentage(gtex_gene_path, percentage=load_percentage)

# Split the data into training and testing sets
train_data, test_data = train_test_split(full_data, test_size=0.2, random_state=42)

# Create dataset instances for training and testing
gene_train_dataset = GeneExpressionDataset(train_data.iloc[:, 1:], train_data.iloc[:, 0])
gene_test_dataset = GeneExpressionDataset(test_data.iloc[:, 1:], test_data.iloc[:, 0])

gene_train_loader = DataLoader(gene_train_dataset, batch_size=64, shuffle=True)
gene_test_loader = DataLoader(gene_test_dataset, batch_size=64, shuffle=False)  # Usually, shuffling is not needed for testing


In [8]:
full_data

,sample_id,TMEM38B,SLC24A3,AXDND1,DUXA,ZCCHC13,FGF18,INPP5D,MAP2K4,BCAR1,...,MARCH10,EVL,CYP1A2,ZNF782,LIMCH1,WDR24,ANGPTL4,UGT2B7,PIPOX,CD1B
0,GTEX-11OC5-0526-SM-5N9EE,2.073820,0.903038,0.275007,0,0,2.025029,5.518850,3.590961,5.004951,...,2.232661,6.223036,0.014355,1.819668,5.989366,3.499527,3.705978,0.042644,0.910733,0
1,GTEX-14BIM-0011-R4b-SM-5S2RK,1.871844,3.390943,0.263034,0,0,0.321928,2.935460,3.970854,4.031219,...,1.021480,7.375300,0.028569,0.739848,3.771886,2.295723,4.958379,0.000000,2.333424,0
2,GTEX-1F75A-0011-R7b-SM-9QEHE,1.778209,3.820690,0.250962,0,0,0.176323,2.157044,4.173927,4.626439,...,2.130931,7.749333,0.000000,0.871844,4.730096,2.769772,3.510962,0.000000,3.099295,0
3,GTEX-1HBPM-2626-SM-9WYU5,2.687061,3.663345,0.344828,0,0,0.389567,3.237258,3.339137,4.860963,...,1.344828,6.072106,0.000000,1.526069,3.952334,3.085765,4.988230,0.000000,2.117695,0
4,GTEX-Q2AI-0226-SM-48U1D,3.218781,1.827819,0.669027,0,0,0.137504,1.735522,3.575312,4.532317,...,0.111031,5.329841,0.000000,1.028569,4.800123,3.431623,2.577731,0.042644,0.485427,0
5,GTEX-ZDTS-1226-SM-4WKGL,3.593354,0.443607,0.495695,0,0,1.378512,1.863938,2.742006,5.080231,...,0.000000,5.884354,0.000000,0.704872,5.429281,2.510962,2.641546,0.042644,0.097611,0


In [9]:
genes, labels = next(iter(gene_train_loader))
genes, labels

(tensor([[2.6871, 3.6633, 0.3448,  ..., 0.0000, 2.1177, 0.0000],
         [3.5934, 0.4436, 0.4957,  ..., 0.0426, 0.0976, 0.0000],
         [1.7782, 3.8207, 0.2510,  ..., 0.0000, 3.0993, 0.0000],
         [3.2188, 1.8278, 0.6690,  ..., 0.0426, 0.4854, 0.0000]]),
 tensor([1, 3, 0, 2]))

In [10]:
test_genes, test_labels = next(iter(gene_test_loader))
test_genes, test_labels

(tensor([[2.0738, 0.9030, 0.2750,  ..., 0.0426, 0.9107, 0.0000],
         [1.8718, 3.3909, 0.2630,  ..., 0.0000, 2.3334, 0.0000]]),
 tensor([0, 1]))

Get original labels

In [11]:
# Convert encoded labels back to original string labels
original_labels = gene_train_dataset.get_original_labels(labels.numpy())
original_labels

array(['GTEX-1HBPM-2626-SM-9WYU5', 'GTEX-ZDTS-1226-SM-4WKGL',
       'GTEX-1F75A-0011-R7b-SM-9QEHE', 'GTEX-Q2AI-0226-SM-48U1D'],
      dtype=object)

# Todo: Add code to load VAE or latent features outputed from the VAE for gtex_gene_path

## Building the model
When defining the model the latent layer must act as a bottleneck of information, so that we ensure that we find a strong internal representation. We initialize the VAE with 1 hidden layer in the encoder and decoder using relu units as non-linearity.

## Training and Evaluation

### Initialize the model, evaluator and optimizer

In [12]:
print(torch.__version__)
print(torch.version.cuda)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f">> Using device: {device}")

1.10.2+cu102
10.2
>> Using device: cpu


In [13]:
train_loader = gene_train_loader
test_loader = gene_test_loader

## Evaluate model